In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?
# Nando

In [ ]:
df_movies = pd.read_csv('../../data/movie.csv')
df_movies.head()
